## Question 1
#### Subroutine to generate a Uniform Distribution

In [280]:
import random

def sample_uniform(n, k):
    samples = []
    collisions = 0
    while len(samples) < k:
        x = random.randrange(0, n, 2)
        if x in samples:
            collisions += 1
        samples.append(x)
    return samples, collisions

n = 10
k = 5
samples, collisions = sample_uniform(n, k)
print(f"Generated {k} samples from Uniform Distribution:")
print(samples)
print(f"Number of collisions: {collisions}")

Generated 5 samples from Uniform Distribution:
[0, 8, 4, 4, 0]
Number of collisions: 2


#### Subroutine to generate far from uniform distribution

In [282]:
def sample_far_uniform(n, epsilon, k):
    p = [(1 + epsilon) / n] * n
    # print(p)
    samples = []
    collisions = 0
    for _ in range(k) :
        i = random.choices(range(n), weights = p)[0]
        if i in samples:
            collisions += 1
        samples.append(i)
        p[i] = p[i]/(1 + epsilon)
        for j in range(n):
            if j != i:
                p[j] *= (1 + epsilon)
        # print(p)
    return samples, collisions

n = 10  # range of the distribution
epsilon = 0.1  # total variation distance from uniform
k = 5  # number of samples to generate

x, c = sample_far_uniform(n, epsilon, k)
print("Samples:", x)
print("Number of collisions:", c)

Samples: [7, 9, 3, 3, 5]
Number of collisions: 1


## Question 2

In [336]:
from math import ceil

def find_s_and_t(n, epsilon, delta):
    s = 10
    while True:
        num_trials = ceil(2/delta) # number of trials needed to achieve 1-delta probability
        
        u_collisions = []
        dfar_collisions = []
        
        for _ in range(num_trials):
            u_samples, u_col = sample_uniform(n, s)
            u_collisions.append(u_col)
            
            dfar_samples, dfar_col = sample_far_uniform(n, epsilon, s)
            dfar_collisions.append(dfar_col)
        
        u_thresh = sorted(u_collisions)[int((delta)*num_trials)] # threshold for U
        dfar_thresh = sorted(dfar_collisions)[int(1-delta*num_trials)] # threshold for Dfar
        
        if u_thresh <= dfar_thresh: # if the thresholds are not compatible, increase s and try again
            s = int(s * 1.1)
        else:
            return s, max(u_thresh, dfar_thresh)

# Example usage
n = 10000
epsilon = 0.2
delta = 0.1

s, t = find_s_and_t(n, epsilon, delta)

print("Sufficient number of samples: ", s)
print("Threshold for collision count: ", t)

Sufficient number of samples:  455
Threshold for collision count:  18


## Question 4
#### Generating a distribution from the dataset

In [337]:
from sklearn.datasets import fetch_california_housing 
import numpy as np 

def generate_data_dist(size):
    n = 10 ** size 
    data = fetch_california_housing(as_frame=True) 
    df = data.frame 
    last_digits = ((df['AveRooms']*n)%n).astype(int).astype(str).str[-3:].astype(int) 
    hist, bin_edges = np.histogram(last_digits, bins=np.arange(0, n+1)) 
    hist = hist/sum(hist) 
    return hist

distribution = generate_data_dist(2)

#### Code to sample from the distribution

In [439]:
def sample_dist_cali(s, dist):
    n = len(dist)
    samples = []
    collisions = 0
    for _ in range(s) :
        i = random.choices(range(n), weights = dist)[0]
        if i in samples:
            collisions += 1
        samples.append(i)
    return samples, collisions

#### Getting the optimal $s$ and $t$ values

In [478]:
from tqdm import tqdm
import numpy as np

def find_s_and_t_cali(n, delta, distribution):
    s = 10

    # pbar = tqdm(total=None)
    while True:
        # pbar.update(÷)
        
        num_trials = ceil(2/delta) # number of trials needed to achieve 1-delta probability
        
        u_collisions = []
        dist_collisions = []
        
        for _ in range(num_trials):
            u_samples, u_col = sample_uniform(n, s)
            u_collisions.append(u_col)
        
            dist_samples, dist_col = sample_dist_cali(s, distribution)
            dist_collisions.append(dist_col)
        
        u_thresh = sorted(u_collisions)[int((1-delta)*num_trials)] # threshold for U
        dist_thresh = sorted(dist_collisions)[int((delta)*num_trials)] # threshold for Dfar
        
        # print(s, sorted(u_collisions), sorted(dist_collisions))
        if u_thresh >= dist_thresh: # if the thresholds are not compatible, increase s and try again
            s = int(s * 1.1)
        else:
            return s, max(u_thresh, dist_thresh)

In [352]:
n = 100
delta = 0.1

s, t = find_s_and_t_cali(n, delta, distribution)

print("Sufficient number of samples: ", s)
print("Threshold for collision count: ", t)

Sufficient number of samples:  46
Threshold for collision count:  14


#### Testing for uniformity using optimal $s$ and $t$ values

In [473]:
dist_samples, dist_col = sample_dist_cali(46, distribution)
print(dist_col)

7


#### Generating the dataset and distribution with pairs of consecutive data points

In [477]:
def generate_data_dist1(size):
    n = 10 ** size 
    data = fetch_california_housing(as_frame=True) 
    df = data.frame 
    last_digits = ((df['AveRooms']*n)%n).astype(int).astype(str).str[-3:].astype(int) 
    pairs = [str(last_digits[i])+str(last_digits[i+1]) for i in range(0, len(last_digits)-1, 2)]
    # print(pairs)
    # print(min(pairs))
    # hist, bin_edges = np.histogram(pairs, bins=100000) 
    hist, bin_edges = np.histogram(last_digits, bins=np.arange(0, 10000)) 

    
    hist = hist/sum(hist) 
    # print(hist)
    return hist, pairs

distribution1, pairs = generate_data_dist1(2)

#### Getting the optimal $s$ and $t$ values for Distribution 1

In [479]:
n = 10000
delta = 0.1

s, t = find_s_and_t_cali(n, delta, distribution1)

# print(distribution1)

print("Sufficient number of samples: ", s)
print("Threshold for collision count: ", t)

Sufficient number of samples:  19
Threshold for collision count:  1


#### Testing for uniformity using optimal $s$ and $t$ values for distribution 1

In [486]:
dist1_samples, dist1_col = sample_dist_cali(19, distribution1)
print(dist1_col)

3


## Question 5
#### Code to generate a single digit

In [487]:
digits_string = "31105691459915887548990328491023590816127834617823647162534182341129803189235612095656391821981324971283950182903589012537865056123985610982365819235471290387512398567501472941928437518293456129487351908245765067589213902191029911238907587927809158160969661028918096312589601283905668568561325869015609128684871437817406932157040041049653792283790435687092433359627943354692330768128237840602462239316825472253747683134255037560420567970649727378716591973343268822116925234046182231692672420526744828624143300730788835207536105663991053290737675565806960294421052278181106136858082678142074494019313525370628268257198532879810915644342460374066977846566984783130966418740309686825181123909028096039085651544520123077715670629736598469123869031308698839585454979426589472743681966406717454009418336141035222046861539555420591446578775295398323456528173651498195051722212244541937857289474509963218345141900253985203724872642248008399816892404699541924169823792409511164985230882728810898575382808002241771703796390265381196049490056167754112222819472363181745649543740543093918068585893476310451469520847579399362687642927116366014297061817851835170391982510970550827389728153728598957562792108117143878863430795438388345140479254480511395615818683663329814837488950853713017084367163035836235234983621310679276846796931502968577693850747882174862562369300819718964235854539079286198437327349924270119363940030103242723415872210596747940910502029529061376169389306044327712162783281819485092971398221149472907392635886108563641164869316145678720901768084252203780476267172138265737195174662256852436093423441142409217224552916282842321448814101828832337338359794981569738203688491383073154475998893241907912101141140796236550130233582144279789105803319480723600013145318401492088703548821121246906770731961642297089028967653702987918213013965705749587590902316956713055780559576300397737639528804706186460976973372470896172089871147070578064066113296357589685795063865647478756798137313733229239934669781095127405984261471206878694574699471604024585266529368365800852049798353708716289049104849629203034499936742090121537467224119062155862050206995164950387116514614512048079265433327473899120495519452383730586936875490987369338745249893788983170539360632399640630486354248287940966228147845857686516724785634508449677162667076930085401740357731155772940426223010451505019628906843969343721438683414897334375541057564543831803730391533551053939226650465980401526627333466763088553568996358873030568005787633359049309985479367259793139580724566882720898340643183488453659322591777266322118259083522803647337080298615362615830407409230983476092734872389750183690476839034872093748320984729347590286032875472039487390802387430592398540329745092834598023475058320464035892370948570234098572380485209385602375967385290735698234050239847590832457032048750298356098437520398465703298765092874309582374058034692384587290384758234750239847534598567340592384059238756092348759023874058092384750923845723904857902348570902384502385070324809573249579023874502387322164570142682905115926825988027816862837276580947081733452581774782369045982174337438221755929376122569559226168584469279121307789944593678572968210385914166711942362102555261947843610331159118684576392810121594767453522893040708066026425846059853387201832714104171555598715835125374971350847049676930907360968187487823232871012788019407058780941236155455893360938660537618652648401634693239912327168032569979585934456298673219994365406558150893016986181822183890378568277030913750977903231677099320865960576778708281616273130736307044089695117936234843468623875603644294039212330839647682064429564527015345821109528860926236784167295924839755944168388297071634593468979594078643825252958583141781864232440424055550010982114742031586859263677422642058047861382736820148070662694651421363593039518040875746154639139227646766835965305806373344085784188833523142744908057536411843350462957614675785205955920535534065623007245082570541548175563250874015696036111356886678120757105277129016029693064008823778987797676231383378752294178572386907177510600153964718292684119580543030003418296427610838961843794844272979170621097791695747326140962144836835918381785218989114178268058758011810963275530313167115484826346606846114686499764428965529688389775929648870882314357329784737042478461860258626862754480923640738542035056438526855277844288903285471691214854924598089264008070745453944838607820301613977146500734308139404168438358105776487936119413880387126917288286263691569790606698655535460026646071042898260165528946703472840044780800862329945082115181206375966959298368040518198038558936402809211535768222808246587521100133254520344834132058690479054393400222556687360824090983786084579282302381354015304210291511798970432169303476648364110053658030727302208707439182686826355154205689470425420277904998675416614448566732831456728027757388539481711079202038216153572254768702222160888473381391152325546201265784308067256202708775408427716685257268771871117661186434208343562772880686804090643538382272955820620132286744888447497033015831156972072822357920345759791138658327833441677739939817011279666063888207127464214240635925212131081480937858108947223752437934393383479645840356002346240290417091946348222751818151765036052310259357391386854313129198143089598273704458337618141304231563725939678321702055791568362317223673497950112283166585722825696889500921203983098682957926225203729648952030343072938740673777039516012144223244901819559534846902797130041186018047077550831714324215327883685277537"
len(digits_string)

5555

In [488]:
import random
def generate_single_digit(q):
    s = 0
    start = random.randint(0, len(digits_string) - q)
    end = start + q
    for i in range(start, end, 1):
        s += int(digits_string[i])
    return s % 10

In [489]:
generate_single_digit(100)

8

#### Generating data of digits and their distribution

In [490]:
def generate_digits_data(q, size_data):
    digits = []
    for i in range(size_data):
        digits.append(generate_single_digit(q))
    return digits

def generate_digit_dist(digits, size):
    n = 10 ** size 
    hist, bin_edges = np.histogram(digits, bins=np.arange(0, n+1)) 
    hist = hist/sum(hist) 
    return hist

In [491]:
size_data = 10000
digits = generate_digits_data(1, size_data)
digit_dist = generate_digit_dist(digits, 1)
print(digits)
# print(digit_dist)

[8, 9, 6, 0, 9, 9, 7, 8, 3, 3, 6, 2, 6, 6, 9, 0, 4, 4, 3, 9, 9, 9, 4, 8, 9, 9, 7, 9, 8, 3, 9, 7, 4, 5, 8, 6, 0, 6, 3, 6, 8, 3, 5, 6, 0, 6, 9, 3, 2, 1, 8, 2, 0, 1, 6, 7, 6, 9, 5, 0, 2, 9, 9, 1, 4, 0, 1, 2, 9, 4, 5, 2, 6, 7, 1, 5, 6, 8, 4, 1, 1, 1, 1, 6, 0, 4, 8, 7, 0, 5, 0, 2, 7, 6, 5, 2, 9, 8, 8, 5, 8, 6, 7, 6, 3, 7, 0, 6, 5, 0, 8, 3, 6, 3, 0, 2, 7, 9, 4, 9, 2, 7, 1, 2, 4, 8, 2, 0, 4, 7, 8, 5, 6, 8, 3, 2, 5, 5, 3, 8, 5, 5, 0, 0, 0, 6, 4, 2, 9, 1, 0, 3, 4, 6, 7, 1, 1, 9, 7, 6, 7, 3, 4, 7, 5, 8, 1, 1, 2, 2, 1, 2, 7, 0, 5, 6, 7, 8, 4, 0, 7, 8, 3, 7, 8, 4, 7, 1, 4, 6, 8, 5, 2, 8, 3, 0, 9, 3, 0, 0, 1, 7, 1, 4, 4, 4, 8, 2, 1, 6, 2, 8, 6, 8, 8, 3, 3, 3, 5, 2, 5, 6, 0, 8, 4, 4, 1, 8, 6, 4, 3, 4, 1, 4, 6, 7, 3, 2, 9, 8, 9, 8, 4, 7, 8, 5, 3, 3, 2, 9, 6, 8, 8, 1, 4, 4, 6, 4, 4, 5, 7, 5, 3, 7, 1, 1, 6, 7, 2, 0, 4, 4, 2, 5, 7, 9, 9, 2, 5, 9, 1, 5, 0, 8, 3, 2, 9, 1, 9, 3, 4, 4, 9, 2, 1, 1, 0, 8, 1, 5, 3, 0, 2, 6, 8, 6, 2, 5, 7, 8, 8, 4, 4, 8, 8, 7, 0, 6, 6, 6, 7, 4, 0, 7, 6, 6, 0, 8, 4, 1, 9, 4, 3, 

#### Sampling from this distribution

In [492]:
def sample_dist_q5(k, dist):
    n = len(dist)
    samples = []
    collisions = 0
    for _ in range(k) :
        i = random.choices(range(n), weights = dist)[0]
        if i in samples:
            collisions += 1
        samples.append(i)
    return samples, collisions

#### Finding Optimal $s$ and $t$ 

In [493]:
from tqdm import tqdm
import numpy as np

def find_s_and_t_q5(n, delta, distribution):
    s = 10

    # pbar = tqdm(total=None)
    while True:
        # pbar.update(÷)
        # print(s)
        num_trials = ceil(2/delta) # number of trials needed to achieve 1-delta probability
        
        u_collisions = []
        dist_collisions = []
        
        for _ in range(num_trials):
            u_samples, u_col = sample_uniform(n, s)
            u_collisions.append(u_col)
        
            dist_samples, dist_col = sample_dist_q5(s, distribution)
            dist_collisions.append(dist_col)
        
        u_sort = sorted(u_collisions)
        dist_sort = sorted(dist_collisions)
        
        u_thresh = u_sort[int((delta)*num_trials)] # threshold for U
        dist_thresh = dist_sort[int((1-delta)*num_trials)] # threshold for Dfar
        
        
        if u_thresh <= dist_thresh: # if the thresholds are not compatible, increase s and try again
            s = int(s * 1.1)
        else:
            return s, max(u_thresh, dist_thresh)

#### Finding $q$ for single digits

In [494]:
from math import ceil
q = 1
size_data = 10000
delta = 0.1

while True:
   digits = generate_digits_data(q, size_data)
   digit_dist = generate_digit_dist(digits, 1)
   s, t = find_s_and_t_q5(len(digit_dist), delta, digit_dist)

   x, c = sample_dist_q5(s, digit_dist)
   print(t, c)
   if(c <= t):
      break
   else:
      q+=1

print(q)

5 4
1


#### Finding $q$ for pairs

In [495]:
def generate_data_dist1(size ):
    n = 10 ** size 
    digits = generate_digits_data(1, n*3)
    pairs = [int(str(digits[i])+str(digits[i+1])) for i in range(0, len(digits)-1, 2)]
    # print(pairs)
    hist, bin_edges = np.histogram(pairs, bins=np.arange(0, n+1)) 

    hist = hist/sum(hist) 
    # print(hist)
    # print(len(hist))
    return hist

distribution2 = generate_data_dist1(2)

In [510]:
from math import ceil
q = 5
size_data = 100
delta = 0.1

while True:
   s, t = find_s_and_t_q5(len(distribution2), delta, distribution2)

   x, c = sample_dist_q5(s, distribution2)
   print(t, c)
   if(c < t):
      break
   else:
      q+=1

print(q)

396 373
5


#### Finding q for triples

In [504]:
def generate_data_dist1(size ):
    n = 10 ** size 
    digits = generate_digits_data(1, n*3)
    pairs = [int(str(digits[i])+str(digits[i+1])+str(digits[i+2])) for i in range(0, len(digits)-2, 3)]
    # print(pairs)
    hist, bin_edges = np.histogram(pairs, bins=np.arange(0, n+1)) 

    hist = hist/sum(hist) 
    # print(hist)
    # print(len(hist))
    return hist

distribution3 = generate_data_dist1(3)

In [ ]:
from math import ceil
q = 1
size_data = 1000
delta = 0.1

while True:
   s, t = find_s_and_t_q5(len(distribution3), delta, distribution3)

   x, c = sample_dist_q5(s, distribution3)
   print(t, c)
   if(c < t):
      break
   else:
      q+=1
   print(q)

print(q)